# Albedo timeseries and maps

## Import packages

In [ ]:
import matplotlib.pyplot as plt
import xarray as xr
from c3s_eqc_automatic_quality_control import diagnostics, download

plt.style.use("seaborn-v0_8-notebook")

## Define Request

In [ ]:
collection_id = "satellite-albedo"
chunks = {"year": 1}

common_request = {
    "format": "zip",
    "variable": "albb_bh",
    "product_version": "v2",
    "month": [f"{month:02d}" for month in range(1, 13)],
    "nominal_day": "10",
    "area": [46.91, 50.75, 46.81, 50.85],
}
requests = {
    "noaa_11": {
        "year": [str(year) for year in range(1990, 1995)],
        "horizontal_resolution": "4km",
        "sensor": "avhrr",
    },
    "noaa_14": {
        "year": [str(year) for year in range(1995, 2002)],
        "horizontal_resolution": "4km",
        "sensor": "avhrr",
    },
    "noaa_17": {
        "year": [str(year) for year in range(2002, 2006)],
        "horizontal_resolution": "4km",
        "sensor": "avhrr",
    },
    "spot": {
        "year": [str(year) for year in range(2006, 2014)],
        "horizontal_resolution": "1km",
        "sensor": "vgt",
    },
    "proba": {
        "year": [str(year) for year in range(2014, 2021)],
        "horizontal_resolution": "1km",
        "sensor": "vgt",
    },
}

## Download and transform

In [ ]:
maps = {}
timeseries = []
for satellite, request in requests.items():
    print(f"{satellite=}")

    maps[satellite] = download.download_and_transform(
        collection_id,
        common_request | request | {"satellite": satellite},
        chunks=chunks,
        transform_chunks=False,
        transform_func=diagnostics.time_weighted_mean,
        drop_variables=["crs"],
    )

    ds = download.download_and_transform(
        collection_id,
        common_request | request | {"satellite": satellite},
        chunks=chunks,
        transform_func=diagnostics.spatial_weighted_mean,
        drop_variables=["crs"],
    )
    timeseries.append(ds)

ds_timeseries = xr.concat(timeseries, "time")

## Plot timeseries

In [ ]:
ds_timeseries["AL_BH_BB"].plot()
plt.grid()